In [1]:
from tqdm.notebook import tqdm, trange

In [2]:
trace_names = ["tiny", "testgen", "ls", "plamap", "g++", "grep"]
trace_name = trace_names[2]
trace_path = "./traces/{}.trace".format(trace_name)
data = []
cycles_since_start = 0
with open(trace_path, "r") as fp:
    for line in fp:
        raw = line.split()
        lors = raw[0]
        addr = int(raw[1], 16)
        pc = int(raw[2], 16)
        cycles_since_last = int(raw[3])
        cycles_since_start += cycles_since_last
        data.append({
            "lors": lors,
            "addr": addr,
            "pc": pc,
            "cycles_since_last": cycles_since_last,
            "cycles_since_start": cycles_since_start
        })

In [3]:
print(len(data))
n = len(data)

299327


In [ ]:
dn = [[-1 for i in range(n)] for j in range(n)]
bt = [[-1 for i in range(n)] for j in range(n)]

maxstride = 0
for i in trange(n):
    dn[i][i] = 1
    for j in range(i):
        for k in range(j+1):
            if data[i]["addr"]-data[j]["addr"]!=data[j]["addr"]-data[k]["addr"]:
                continue
            if dn[j][k] == -1:
                continue
            if dn[j][k]+1 > dn[i][j]:
                dn[i][j] = dn[j][k]+1
                maxstride = max(maxstride, dn[i][j])
                bt[i][j] = k

In [ ]:
strides = []
for i in trange(n):
    for j in range(i+1):
        if dn[i][j] == -1:
            continue
        strides.append({"i": i, "j": j, "dn": dn[i][j]})
strides.sort(key=lambda i:i["dn"], reverse=True)
print(len(strides))

In [ ]:
def print_stride(num):
    print(strides[num])
    a = strides[num]["i"]
    b= strides[num]["j"]
    print("Longest stride:")
    while bt[a][b] != -1:
        c = bt[a][b]
        a = b
        b = c
        print("{}: type={}, addr={}, pc={}".format(a, data[a]["lors"], data[a]["addr"], data[a]["pc"]))
        
print_stride(1000)

In [ ]:
from collections import Counter
stride_lens = [x["dn"] for x in strides]
print(len(stride_lens))
print(dict(Counter(stride_lens)))